In [ ]:
import os
import yaml
import time
import torch
import random
import numpy as np

from torch.utils import data

from tensorboardX import SummaryWriter

SEED = 1337

def train(config, writer, logger):
    # setup seeds
    torch.manual_seed(config["seed"])
    torch.cuda.manual_seed(config["seed"])
    np.random.seed(config["seed"])

    # setup device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # setup augmentations
    ## ah todo
    
    # setup dataloader
    